<a href="https://www.kaggle.com/code/pietrocaforio/unimodal-ct-training-kaggle?scriptVersionId=199214851" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 354 (delta 88), reused 71 (delta 26), pack-reused 208 (from 1)
Receiving objects: 100% (354/354), 5.21 MiB | 25.41 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

from sklearn.utils.class_weight import compute_class_weight

### Train ResNet model

In [7]:
#https://github.com/mathiaszinnen/focal_loss_torch/tree/main
!pip install focal_loss_torch

In [8]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [9]:
from focal_loss.focal_loss import FocalLoss

def train(model,config, run_name=None):
    wandb.init(
        # set the wandb project where this run will be logged
        project="unimodal_ct_training",
        name = run_name,
        # track hyperparameters and run metadata
        config=config
    )
    if config["class_weights"] is not None: 
        config["class_weights"] = torch.tensor(config["class_weights"], dtype=torch.float).to(device) 
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
    if config["focal_loss"] is not None:
        
        criterion = FocalLoss(gamma = config["focal_loss"])
    else:
        criterion = nn.CrossEntropyLoss(weight = config["class_weights"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=config["early_stop_patience"], verbose=True)
    
    # Training loop
    num_epochs = config["epochs"]
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_correct = 0
        total = 0
        correct_per_class = [0, 0, 0]  # For G1, G2, G3
        total_per_class = [0, 0, 0]  # For G1, G2, G3
        for batch in train_loader:
            frames = batch['frame'].float().to(device)
            labels = batch['label'].long().to(device)
            
            optimizer.zero_grad()
            outputs = model(frames)
            if config["focal_loss"]:
                softmax = torch.nn.Softmax(dim=-1)
                loss = criterion(softmax(outputs.logits), labels)
            else:
                loss = criterion(outputs.logits, labels)

            
            _, predicted = torch.max(outputs.logits, 1)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()
            total += labels.size(0)
            running_loss += loss.item()
            
            # Calculate accuracy per class
            for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                total_per_class[i] += (labels == i).sum().item()

        train_accuracy = 100 * train_correct / total
        class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")
        wandb.log({"Train Accuracy": train_accuracy, "Train loss": running_loss/len(train_loader), "G1_TrainAcc":class_accuracy[0], "G2_TrainAcc":class_accuracy[1], "G3_TrainAcc":class_accuracy[2]})

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        # Initialize counters for each class (G1, G2, G3)
        correct_per_class = [0, 0, 0]  # For G1, G2, G3
        total_per_class = [0, 0, 0]  # For G1, G2, G3

        with torch.no_grad():
            for batch in val_loader:
                frames = batch['frame'].float().to(device)
                labels = batch['label'].long().to(device)

                outputs = model(frames)
                
                if config["focal_loss"]:
                    softmax = torch.nn.Softmax(dim=-1)
                    loss = criterion(softmax(outputs.logits), labels)
                else:
                    loss = criterion(outputs.logits, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Calculate accuracy per class
                for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                    correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                    total_per_class[i] += (labels == i).sum().item()
        scheduler.step(val_loss)
        # Compute total accuracy and per-class accuracy
        total_accuracy = 100 * correct / total
        class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
        print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
        print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
        # log metrics to wandb
        wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
        early_stopping(val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
    wandb.finish()  

In [10]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification
from sklearn.model_selection import StratifiedGroupKFold #For crossvalidation

In [11]:
train_dataset = UnimodalCTDataset(split='all',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )

In [12]:
#print(f"Training set stats:{train_dataset.stats()}")
#print(f"Validation set stats:{val_dataset.stats()}")

In [13]:
"""
labels = []
for sample in train_dataset:
    labels.append(sample["label"])
labels = np.array(labels)
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
"""

'\nlabels = []\nfor sample in train_dataset:\n    labels.append(sample["label"])\nlabels = np.array(labels)\nclass_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)\n'

In [14]:
#print(class_weights)

In [15]:
#train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/oversampling57patientscptacpda/processed_oversampling/" )
#val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/oversampling57patientscptacpda/processed_oversampling/")

#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [16]:
#print(f"Training set stats:{train_dataset.stats()}")
#print(f"Validation set stats:{val_dataset.stats()}")

In [17]:
#total = train_dataset.stats()["length"]
#most_frequent = max(train_dataset.stats()["class_frequency"].values())
#freq_dict = train_dataset.stats()["class_frequency"]
#target_volume_depth= {}
#for index in freq_dict.keys():
#    target_volume_depth[index] = int((total/3 ) * most_frequent / freq_dict[index])
#print(target_volume_depth)

### Resnet-50

In [18]:
#model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
#model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [19]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [20]:
"""
config={
    "learning_rate": 1e-4,
    "architecture": "microsoft/resnet-50 new1",
    "epochs": 100,
    "weight_decay": 1e-4,
    "reduce_lr_factor": 0.2,
    "patience": 10,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])
"""

'\nconfig={\n    "learning_rate": 1e-4,\n    "architecture": "microsoft/resnet-50 new1",\n    "epochs": 100,\n    "weight_decay": 1e-4,\n    "reduce_lr_factor": 0.2,\n    "patience": 10,\n    "class_weights": class_weights\n    }\ntrain(model, config, run_name = config["architecture"])\n'

### Resnet-18

In [21]:
"""
config={
    "learning_rate": 1e-7,
    "architecture": "microsoft/resnet-18",
    "run_name": "microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS",
    "epochs": 800,
    "weight_decay": 1e-6,
    "reduce_lr_factor": 0.25,
    "patience": 20,
    "early_stop_patience": 40,
    "class_weights": None,
    "focal_loss": 2
    }

"""




'\nconfig={\n    "learning_rate": 1e-7,\n    "architecture": "microsoft/resnet-18",\n    "run_name": "microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS",\n    "epochs": 800,\n    "weight_decay": 1e-6,\n    "reduce_lr_factor": 0.25,\n    "patience": 20,\n    "early_stop_patience": 40,\n    "class_weights": None,\n    "focal_loss": 2\n    }\n\n'

### Resnet-34

In [22]:
#model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
#model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [23]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [24]:

config={
    "learning_rate": 5e-8,
    "architecture": "microsoft/resnet-34",
    "run_name": "microsoft/resnet-34 NOOVERSAMPLING FOCALLOSS",
    "epochs": 800,
    "weight_decay": 1e-5,
    "reduce_lr_factor": 0.25,
    "patience": 20,
    "early_stop_patience": 40,
    "class_weights": None,
    "focal_loss": 5
    }




In [25]:
from torch.utils.data import DataLoader, Subset
from collections import Counter 

k_folds = 3
batch_size = 32
# Initialize the k-fold cross validation
#group fold in order to group indices by patient_id so that we don't introduct frames of the same patient in the train set and test set
gkf = StratifiedGroupKFold(n_splits=k_folds) 
patient_ids = [info.split("/")[0].split("_")[0] for info in train_dataset.items]
labels = [train_dataset.labels[patient_id] for patient_id in patient_ids]
indices = list(range(len(patient_ids)))
# Loop through each fold
for fold, (train_idx, test_idx) in enumerate(gkf.split(indices, labels, groups=patient_ids)):
    print(f"Fold {fold + 1}")
    print("-------")
    # Define the data loaders for the current fold
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, test_idx)
    train_loader = DataLoader(
        dataset=train_subset,
        batch_size=batch_size,
        shuffle = True
    )
    val_loader = DataLoader(
        dataset=val_subset,
        batch_size=batch_size,
        shuffle = False,
    )
    train_ids = []
    train_labels = []
    for frame in train_loader:
        train_ids.extend(frame["patient_id"])
        train_labels.extend(frame["label"])
    print(f"TRAIN LABELS IN FOLD {fold + 1 }: {Counter(train_labels)}")
    val_ids = []
    val_labels = []
    for frame in val_loader:
        val_ids.extend(frame["patient_id"])
        val_labels.extend(frame["label"])
    print(f"VAL LABELS IN FOLD {fold + 1}: {Counter(val_labels)}")
    print(len(set(train_idx) - set(val_ids)))
    print(len(set(train_ids)))
    assert len(set(train_ids) - set(val_ids)) == len(set(train_ids))
    config["run_name"] = f"{config['run_name']} fold:{fold + 1}"
    #Prepare model
    model = ResNetForImageClassification.from_pretrained(config["architecture"])
    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    #Train model
    train(model, config, run_name = config["run_name"])


Fold 1
-------
TRAIN LABELS IN FOLD 1: Counter({tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(0): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(0): 1, tensor(2): 1, 

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241005_124410-a8v34gxb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34 NOOVERSAMPLING FOCALLOSS fold:1
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/a8v34gxb


Epoch 1, Loss: 0.261243654808528
Validation Loss: 0.3421626041332881, Total Accuracy: 34.13%
Accuracy per class - G1: 26.42%, G2: 49.35%, G3: 0.98%
Validation loss decreased (inf --> 11.291366).  Saving model ...
Epoch 2, Loss: 0.24569568166535505
Validation Loss: 0.3571903666086269, Total Accuracy: 30.89%
Accuracy per class - G1: 24.53%, G2: 44.59%, G3: 0.98%
EarlyStopping counter: 1 out of 40
Epoch 3, Loss: 0.24163552426116566
Validation Loss: 0.35861100459640677, Total Accuracy: 31.46%
Accuracy per class - G1: 37.74%, G2: 44.88%, G3: 0.00%
EarlyStopping counter: 2 out of 40
Epoch 4, Loss: 0.2316569167543465
Validation Loss: 0.36068423600359395, Total Accuracy: 32.51%
Accuracy per class - G1: 30.19%, G2: 46.61%, G3: 0.98%
EarlyStopping counter: 3 out of 40
Epoch 5, Loss: 0.21773689880337513
Validation Loss: 0.3584372482516549, Total Accuracy: 30.89%
Accuracy per class - G1: 41.51%, G2: 43.58%, G3: 0.33%
EarlyStopping counter: 4 out of 40
Epoch 6, Loss: 0.20815161794004305
Validation 

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▇▆█▇▇▅▃▃▅▅▃▂▄▃▃▆▄▂▃▄▃▄▂▃▄▂▄▂▂▃▁▂▂▂▂▂▃▁▃▂
wandb:     G1_TrainAcc ▁▁▁▂▂▃▃▄▅▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████████████
wandb:          G2_Acc █▇▇▅▆▅▅▅▄▄▄▃▃▃▄▃▃▄▃▄▂▂▂▂▂▂▂▂▂▁▂▂▃▁▂▁▃▁▁▂
wandb:     G2_TrainAcc ▁▂▂▃▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇█▇██████████████████
wandb:          G3_Acc ▁▂▁▂▁▂▂▂▂▄▅▄▃▄▃▃▅▆▆▆▆▄▆▆▆▇▇▇▆▆▇▇█▇▆▇█▇▇▇
wandb:     G3_TrainAcc ▁▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇██████████████
wandb:  Total Accuracy █▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▄▃▃▃▃▂▂▂▃▃▂▂▂▁▂▁▁▁▁▁▂
wandb:  Train Accuracy ▁▂▂▃▃▃▃▄▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████
wandb:      Train loss █▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: Validation Loss █▄▆▇▄▅▅▄▆▆▆▅▃▅▅█▆▆▇▃▃▁▃▂▂▂▁▄▆▃▃▃▇▆▄▄▆▇▆▅
wandb: 
wandb: Run summary:
wandb:          G1_Acc 5.40541
wandb:     G1_TrainAcc 96.71362
wandb:          G2_Acc 54.04624
wandb:     G2_TrainAcc 98.12139
wandb:          G3_Acc 7.32484
wandb:     G3_TrainAcc 93.51702
wandb:  Total Accuracy 37.1

Fold 3
-------
TRAIN LABELS IN FOLD 3: Counter({tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(0): 1, tensor(0): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(0): 1, tensor(1): 1, tensor(0): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(0): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(2): 1, tensor(0): 1, tensor(2): 1, tensor(1): 1, tensor(1): 1, tensor(1): 1, tensor(2): 1, tensor(1): 1, tensor(0): 1, tensor(2): 1, 

wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241005_133741-8b50ad0x
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34 NOOVERSAMPLING FOCALLOSS fold:1 fold:2 fold:3
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/8b50ad0x


Epoch 1, Loss: 0.30328107969974405
Validation Loss: 0.37625305888217847, Total Accuracy: 22.29%
Accuracy per class - G1: 3.12%, G2: 5.21%, G3: 70.10%
Validation loss decreased (inf --> 13.921363).  Saving model ...
Epoch 2, Loss: 0.28545033331237624
Validation Loss: 0.3468014490161393, Total Accuracy: 25.22%
Accuracy per class - G1: 5.62%, G2: 6.51%, G3: 76.85%
Validation loss decreased (13.921363 --> 12.831654).  Saving model ...
Epoch 3, Loss: 0.2775739589289053
Validation Loss: 0.3545464627646111, Total Accuracy: 23.06%
Accuracy per class - G1: 4.38%, G2: 6.51%, G3: 69.45%
EarlyStopping counter: 1 out of 40
Epoch 4, Loss: 0.26582370231400676
Validation Loss: 0.3417025524619463, Total Accuracy: 26.08%
Accuracy per class - G1: 5.62%, G2: 7.53%, G3: 77.81%
Validation loss decreased (12.831654 --> 12.642994).  Saving model ...
Epoch 5, Loss: 0.2570168380417041
Validation Loss: 0.3292239808754341, Total Accuracy: 24.44%
Accuracy per class - G1: 5.62%, G2: 6.51%, G3: 73.95%
Validation los

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▇▅█▇▆▅▅▅▅▅▅▂▅▂▃▃▄▂▂▁▂▃▃▁▁▁▂▁▁▁▁▂▁▁▁▁▁▂▁▁
wandb:     G1_TrainAcc ▁▁▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████
wandb:          G2_Acc ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▆▅▆▆▆▅▆▇▇▇▇█▇█▇▇██
wandb:     G2_TrainAcc ▁▁▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████████████
wandb:          G3_Acc ▇█▇▇▇▆▆▆▆▅▆▆▅▅▅▅▅▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▁▂▂
wandb:     G3_TrainAcc ▁▂▁▂▂▃▄▄▄▄▅▅▅▆▅▆▇▆▆▇▇▇▇▇▇██▇████████████
wandb:  Total Accuracy ▁▂▁▁▂▂▂▂▂▂▁▂▂▂▃▃▄▅▅▅▅▆▆▆▆▇▆▆▇▇▇▇█▇▇█▇▇▇█
wandb:  Train Accuracy ▁▁▁▂▂▂▃▃▃▄▅▅▅▅▅▆▆▇▇▇▇▇█▇████████████████
wandb:      Train loss ██▇▇▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: Validation Loss █▆▆▆▆▆▅▅▅▅▄▃▃▄▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.625
wandb:     G1_TrainAcc 100
wandb:          G2_Acc 58.17656
wandb:     G2_TrainAcc 99.35018
wandb:          G3_Acc 30.54662
wandb:     G3_TrainAcc 99.03226
wandb:  Total Accuracy 42.85714
w